In [2]:
from nltk import pos_tag, word_tokenize

In [8]:
import json
import contractions
import numpy as np
import tensorflow as tf
import keras
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from keras_preprocessing.text import tokenizer_from_json
from keras.models import Model, Sequential #This works apparently and I have no idea why
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
    
#Parameters of Tokenizer
vocab_size = 1000
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

stop_words = set(stopwords.words('english'))

intent_recognition_model = keras.models.load_model('intent_recognition_model.keras')
probability_model = Sequential([intent_recognition_model, tf.keras.layers.Softmax()])

def getTokens(user_prompt): #Accepts string
    tokens = user_prompt
    tokens = contractions.fix(user_prompt)
    #print(f"User Prompt: {user_prompt}")
    tokens = tokenizer.texts_to_sequences([tokens])
    tokens = pad_sequences(tokens, maxlen=max_length, padding=padding_type) #Scale the input for the model
    #print(f"Tokens: {tokens}")
    return tokens
    

def getIntent(prompt): #Accepts string
    tokens = getTokens(prompt)
    probabilities = probability_model.predict(tokens)
    #print(f"Probabilities: {probabilities}")
    predicted_label = np.argmax(probabilities)
    #print(f"Tokens: {tokens}")
    return predicted_label

token_ban_list = ["available"]
#We must identify nouns for keyword extraction
def getKeywords(user_prompt): #Obtain nouns from tokens using POS Tagger
    tokens = word_tokenize(user_prompt)
    tags = pos_tag(tokens)
    extracted_keywords = []
    possible_keyterm = ""
    for index in range(len(tags)): #Try to get key terms based on POS tags of words
        if(tags[index][1] in ["JJ", "VBN", "NN", "NNS", "NNP", "NNPS"] and tags[index][0] not in token_ban_list):
            possible_keyterm = possible_keyterm + f"{tags[index][0]} "
        else:
            extracted_keywords.append(possible_keyterm)
            possible_keyterm = ""
    extracted_keywords.append(possible_keyterm)
    extracted_keywords = [word.strip() for word in extracted_keywords if word != ""]
    #print(tokens)
    print(tags)
    print(f"extracted_keywords: {extracted_keywords}")

    
sample_sentence = "Is Ascorbic Acid available?"
getKeywords(sample_sentence)

[('Is', 'VBZ'), ('Ascorbic', 'NNP'), ('Acid', 'NNP'), ('available', 'JJ'), ('?', '.')]
extracted_keywords: ['Ascorbic Acid']
